## Big Data Visualization
Valkyrie and Icesat 2 data are by nature big data sets that require some special considerations when working with it. The main constraint if you don't have a super computer is memory. The average granule size is in the 10s of MB for IceSat 2 and could be Gigabytes in Valkyrie depending on the order/subsetting. 

This is when libraries like Dask, Vaex and others come into play. This notebook will show you how to use some basic plotting techniques using matpplotlib and geopandas + Vaex to work effectively with lidar data from Valkyrie and ATL data from IceSat 2.


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import glob
import geopandas
import pandas as pd
import h5py
import vaex
import dask.dataframe as dd
import dask.array as da
import numpy as np

## Loading the data

In [ ]:
%%time

f = h5py.File('data/atm1b_data_2020-07-11T20-39.hdf5', 'r')
print(list(f.keys()))
df_data = {
    'latitude': f['latitude'],
    'longitude': f['longitude'],
    'elevation': f['elevation'],
    'time': pd.to_datetime(f['utc_datetime'])
}
df = pd.DataFrame(data=df_data)
df.describe()

In [ ]:
%%time
df = vaex.open('data/atm1b_data_2020-07-11T20-39.hdf5')
# We're parsing the utc_datetime from Valkyrie into a data type that Vaex understands.
df['date'] = df.utc_datetime.values.astype('datetime64[ns]')
my_df = df['longitude', 'latitude', 'elevation', 'date']
# vaex.vrange() is like numpy.arange but uses 0-memory no matter the length.
df.add_column('index', vaex.vrange(0, len(df)))
# We are going to create a "decimated" dataframe with only 1/100 of the size of the original to plot the big picture faster.
df_decimated = df[(df.index % 100 == 0)]
my_df.describe()

## Visualizing the big picture

In [ ]:
my_df.widget.heatmap(my_df.longitude, 
               my_df.latitude,
               what=vaex.stat.mean(my_df.elevation),
               shape=512, 
               figsize=(10,6),
               limits='minmax',
               colormap='plasma')

In [ ]:
%matplotlib widget
import vaex
from ipywidgets import widgets
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

plt.figure(figsize=(10,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0)) 
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-50, -40, 60, 90], ccrs.PlateCarree())
plt.scatter(df_decimated.longitude.values,
            df_decimated.latitude.values,
            c=df_decimated.elevation.values,
            cmap='viridis',
            vmin=100,vmax=1000,
            transform=ccrs.PlateCarree())
plt.colorbar(label='elevation', shrink=0.5,extend='both')

In [ ]:
%matplotlib widget
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(70, 70)

def plot_func(alontrack):
    step = 5000 # same as density
    m = int(alontrack * step)
    ax.clear()
    ax.scatter(df.longitude.values[m:m+step],
               df.latitude.values[m:m+step],
               df.elevation.values[m:m+step],
               c=df.elevation.values[m:m+step],
               cmap='viridis', s=1)
    ax.axis('tight')
    
    

interact(plot_func, alontrack = widgets.FloatSlider(value=0,
                                                    description='Along Track Steps',
                                                    min=0,
                                                    max=90,
                                                    step=0.3,
                                                    layout={'width': '100%'}))

## Time Series